In [2]:
import platform
import requests
import os
import time
import json
import psutil
import platform
import shutil
import pynvml
import multiprocessing

In [3]:
class Logger():

    def __init__(self) -> None:
        self.__pid = os.getpid()
        self.__loadSavePath = "http://127.0.0.1:5560/ml_client/client/loadSavePath"
        self.__noticeExperiment = "http://127.0.0.1:5560/ml_client/client/noticeExperiment"
        self.__noticeRun = "http://127.0.0.1:5560/ml_client/client/noticeRun"


        print("验证代理客户端状态 \n")

        try:
            # resp  = requests.get(utl=self.__loadSavePath)
            print("开发阶段跳过校验 \n")
        except:
            raise BaseException("代理客户端未启动,请重新检查\n")
        else:
            print("代理客户端验证通过 \n")
            self.__login = True
            self.__location = "."
            self.__savedir = "code"
            self.__codedir = "src"
            self.__srcignore = "datasets"


        i = 0
        while os.path.exists(f"run-{i}"):
            i+=1
        self.__uid = f"run-{i}"


        if os.path.exists(self.__location) :
            print(f"创建本次运行记录，记录位置：{self.__location}/{self.__uid} \n")
            os.makedirs(self.__location+f"/{self.__uid}",mode=0o777)
        else:
            print(f"目录 {self.__location} 不存在,将自动创建 \n")
            os.makedirs(self.__location+f"/{self.__uid}",mode=0o777)
        self.__trainning_status  = {"epoch":[]}

        self.Watcher()

        

    def Watcher(self):

        def isProcessRunning(main_pid:int) ->bool:
            try:
                ps = psutil.Process(pid=main_pid)
                return ps.is_running
            except:
                return False
   
        # 此函数需要子进程执行
        def WatchCPU(main_pid:int)->None:
            os.makedirs(self.__location+f"/{self.__uid}/watcher",mode=0o777)
            with open(f"{self.__location}/{self.__uid}/watcher/cpu.log","a") as f:
                running = isProcessRunning(main_pid=main_pid)
                while running:
                    running = isProcessRunning(main_pid=main_pid)
                    # f.write( f"time: { time.strftime('%Y-%m-%d %X', time.localtime())}, cpu_percent: {psutil.Process(pid=self.__pid).cpu_percent()} {psutil.Process(pid=self.__pid).memory_info().rss} \n")
                    f.write(str({"time":time.strftime('%Y-%m-%d %X', time.localtime()),"cpu_percent":psutil.Process(pid=self.__pid).cpu_percent(),"memory":psutil.Process(pid=self.__pid).memory_info().rss})+"\n")
                    f.flush()
                    time.sleep(5)
                    
        watch_cpu = multiprocessing.Process(target=WatchCPU ,daemon=True, args=(self.__pid,))
        watch_cpu.start()

        def WatchGPU(main_pid:int)->None:

            if not os.path.exists(self.__location+f"/{self.__uid}/watcher"):
                os.makedirs(self.__location+f"/{self.__uid}/watcher",mode=0o777)

            pynvml.nvmlInit()

            device_count = pynvml.nvmlDeviceGetCount()

            with open(f"{self.__location}/{self.__uid}/watcher/gpu.log","w") as f:
                running = isProcessRunning(main_pid=main_pid)
                while running:
                    running = isProcessRunning(main_pid=main_pid)
                    device_status =[]

                    for i in range(device_count):
                       
                        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
                    
                       
                        gpu_percent = pynvml.nvmlDeviceGetUtilizationRates(handle)
                        gpu_memory = pynvml.nvmlDeviceGetMemoryInfo(handle)

                        device_status.append({"time":time.strftime('%Y-%m-%d %X', time.localtime()),"gpu_percent":gpu_percent.gpu,"gpu_memory":gpu_memory.used})

                    
                    f.write(str(device_status)+"\n")
                    f.flush()
                    time.sleep(5)

        watch_gpu = multiprocessing.Process(target=WatchGPU,daemon=True, args=(self.__pid,))
        watch_gpu.start()

    def Start(self,info:dict) ->None:
        
        try:
            now = time.strftime("%Y-%m-%d %X", time.localtime())
            print(f"运行开始时间：{now} \n")
            self.__trainning_status["start_at"] = now

            device_list =[]

            try:
                print("获取Nvidia显卡信息......")
                pynvml.nvmlInit()
                device_count=pynvml.nvmlDeviceGetCount()

                for i in range(device_count):
                    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
                    device_list.append(str(pynvml.nvmlDeviceGetName(handle)))

            except:
                print("未获取到Nvidia显卡信息......")
            
            self.__NvidiaGPU= device_list

            try:
                self.__osinfo = {
                "plantform":platform.platform(),
                "system":platform.system(),
                "python_version":platform.python_version(),
                "architecture":platform.architecture()[0],
                "processor":platform.processor(),
                "uname":str(platform.uname()),
                "cpu_logical_count":psutil.cpu_count(),
                "cpu_count": psutil.cpu_count(logical=False),
                "total_memory": psutil.virtual_memory().total /100000,
                "active_memory": psutil.virtual_memory().active /100000,
                "available_memory": psutil.virtual_memory().available /100000,
                "total_swap_memory":psutil.swap_memory().total /100000,
                "nvidia_gpu_info":str(self.__NvidiaGPU)
                
            }
            except:
                 raise BaseException("系统信息采集失败")
            
            
            with open(f"{self.__location}/os_info.json","w") as f:
                f.write(str(json.dumps(self.__osinfo,indent=2)))
                f.flush()

            with open(self.__location+"/"+self.__uid+"/super_arg.json",mode="w") as f:
                f.write(f"{json.dumps(info)}")
                f.flush()
            return

        except:
            raise BaseException("日志实例启动失败\n")
        
    def SaveFile(self,path_list:list[str]) ->None:
        for path in path_list:
            if os.path.exists(path):
                print(path)
        return
    
        
    def EpochInit(self) ->None:

        return

    def EpochLog(self,info:dict) ->None:
        try:
            this_epoch = info
            self.__trainning_status["epoch"].append(this_epoch)

            with open(f"{self.__location}/{self.__uid}/results.json","w") as f:
                f.write(str(json.dumps(self.__trainning_status["epoch"],indent=2)))
                f.flush()
            data ={
                    "plantform":platform.platform(),
                    "system":platform.system(),
                    "python_version":platform.python_version(),
                    "architecture":platform.architecture()[0],
                    "processor":platform.processor(),
                    "uname":str(platform.uname()),
                    "cpu_logical_count":psutil.cpu_count(),
                    "cpu_count": psutil.cpu_count(logical=False),
                    "total_memory": psutil.virtual_memory().total /100000,
                    "active_memory": psutil.virtual_memory().active /100000,
                    "available_memory": psutil.virtual_memory().available /100000,
                    "total_swap_memory":psutil.swap_memory().total /100000,
                    "nvidia_gpu_info":str(self.__NvidiaGPU)
                }

            with open(f"{self.__location}/{self.__uid}/os_info.json","w") as osfile:
                osfile.write(str(json.dumps(data,indent=2)))
        except:
            raise BaseException("循环日志采集失败")
        return

    def End(self) ->None:
        now = time.strftime("%Y-%m-%d %X", time.localtime())
        print(f"运行结束时间：{now}\n")
        self.__trainning_status["end_at"] = now
        with open(self.__location+f"/{self.__uid}"+"/finish.tag",mode="a") as f:
            f.write(f"{now} | {self.__uid} \n")
        
        self.SaveCode()
        return
    
    def SaveCode(self)->None:
        try:
            shutil.copytree(src=self.__codedir,dst=self.__savedir,dirs_exist_ok=True)
        except:
            raise BaseException("备份代码失败,检查代码路径 \n")
        
        else:
            return
        
    def SuperArg(self,info:dict)->None:
        with open(self.__location+"/"+self.__uid+"/super_arg.json",mode="w") as f:
            f.write(f"{json.dumps(info)}")
        return

    def ShowStatus(self) -> str:
        return json.dumps(self.__trainning_status,indent=2)
    # 


In [4]:
# 创建实例
# 1.实例创建时请求本地客户端
# 确认客户端启动后（三次），继续执行；
# 否则抛出错误，程序终止。
log = Logger()


验证代理客户端状态 

开发阶段跳过校验 

代理客户端验证通过 

创建本次运行记录，记录位置：./run-3 



In [5]:

# 开始任务，提供
# 1.记录超参（常量）
# 2.记录运行代码的软件、硬件信息
# 3.通知客户端，实验开始
# 4.开启cpu、内存记录信息进程
# 5.开启显卡核心、显存信息记录进程
log.Start(info={"learnning_rate":0.002,"epoch":10}) 

i=0
# 模拟epoch开始循环
while i<10 :

    # 1. 通知客户端一个批处理开始
    log.EpochInit()

    # 模拟任务代码
    time.sleep(1)

    # 循环结束
    # 1. 为运行id文件夹添加finish.tag
    # 2. 通知客户端，一个批处理结束
    # 3. 记录回调中要记录的参数
    log.EpochLog({"acc":0.83,"loss":0.02})
    i+=1

# 结束任务
# 1.记录运行代码
# 2.通知客户端结束实验
log.End()

# 工具方法：打印日志类状态
log.ShowStatus()

运行开始时间：2023-10-17 17:23:30 

获取Nvidia显卡信息......


运行结束时间：2023-10-17 17:23:40



'{\n  "epoch": [\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    },\n    {\n      "acc": 0.83,\n      "loss": 0.02\n    }\n  ],\n  "start_at": "2023-10-17 17:23:30",\n  "end_at": "2023-10-17 17:23:40"\n}'